## The Saltenis estimator

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the test functions

In [ ]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

def A1(sm):
    return pd.Series([np.prod(sm.iloc[:,:j+1],axis=1)*(-1)**(j+1) for j in range(k)]).sum()

def A2(sm):
    return pd.Series([(np.abs(4*sm[j]-2)+a2[j])/(1+a2[j]) for j in range(k)]).product()

def A2b(sm,sn):
    return pd.Series([(np.abs(4*(sm[j]+sn[j]-np.modf(sm[j]+sn[j])[1])-2)+a2[j])/(1+a2[j]) for j in range(k)]).product()

def B1(sm):
    return pd.Series([(k-sm[j])/(k-0.5) for j in range(k)]).product()
        
def B2(sm):
    return ((1+1/k)**k)*pd.Series([sm[j]**(1/k) for j in range(k)]).product()
        
def B3(sm):
    return pd.Series([(np.abs(4*sm[j]-2)+b3[j])/(1+b3[j]) for j in range(k)]).product()
        
def C1(sm):
    return pd.Series([np.abs(4*sm[j]-2) for j in range(k)]).product()
        
def C2(sm):
    return sm.product(axis=1)*2**k

functions = [A1, A2, B1, B2, B3, C1, C2]

### And the analytical values

In [ ]:
def Ek2(k):
    return((1/6)*(1-(1/3)**k)+(4/15)*((-1)**(k+1)*(1/2)**k+(1/3)**k))

def V(k):
    return((1/10)*(1/3)**k+(1/18)-(1/9)*(1/2)**(2*k)+(-1)**(k+1)*(2/45)*(1/2)**k)

for j in range(0, k):
    def Ej(j):
        return((1/6)*(1-(1/3)**(j))+(4/15)*((-1)**(j+1)*(1/2)**(j)+(1/3)**(j)))
              
    def T1(j):
        return((1/2)*((1/3)**(j-1)*(1-(1/3)**(k-j))))

    def T2(j):
        return((1/2)*((1/3)**j-(1/3)**k))
    
    def T3(j):
        return((3/5)*(4*(1/3)**(k+1)+(-1)**(j+k+1)*(1/2)**(k-j-2)*(1/3)**(j+1)))
    
    def T4(j):
        return((1/5)*((-1)**(j+2)*(1/3)*(1/2)**(j-2)-4*(1/3)**(j+1)))
    
    def T5(j):
        return((1/5)*((-1)**(k+1)*(1/3)*(1/2)**(k-2)+(-1)**(k+j)*(1/3)**(j+1)*(1/2)**(k-j-2)))
    
def A1ST(j):
    return((Ek2(k)-Ej(j)-(1/4)*(T1(j)-2*T2(j)+T3(j))-T4(j)-T5(j))/V(k))

#ÁEA2

def VjA2(j):
    return((1/3)/(1+a2[j])**2)

def VA2(j):
    productA2 = []
    for j in range(0, k):
        productA2.append(1+VjA2(j))
    return np.product(productA2)-1

def VnA2(j):
    return((VA2(j)+1)/(1+VjA2(j)))

def VTjA2(j):
    return VjA2(j)*(VnA2(j))

def A2ST(j):
    return(VTjA2(j)/VA2(j))

#AEB1

def q(j):
    return(12*(k-0.5)**2)
        
def B1ST(j):
    return((q(j)+1)**k/((q(j)+1)*((q(j)+1)**k-q(j)**k)))

#AEB2

def B2ST(j):
    return((k+1)**(2*k-2)/(((k+1)**(2*k)-(k**2+2*k)**k)))

#AEB3

def VjB3(j):
    return((1/3)/(1+b3[j])**2)

def VB3(j):
    productB3 = []
    for j in range(0, k):
        productB3.append(1+VjB3(j))
    return np.product(productB3)-1

def VnB3(j):
    return((VB3(j)+1)/(1+VjB3(j)))

def VTjB3(j):
    return VjB3(j)*(VnB3(j))

def B3ST(j):
    return(VTjB3(j)/VB3(j))

#AEC1

def C1ST(j):
    return 4**(k-1)/(4**k-3**k)

#AEC2
        
def C2ST(j):
    return 4**(k-1)/(4**k-3**k)

def create_dict(key, values):
    return dict(zip(key, values))

analyticalValues = [A1ST,A2ST,B1ST,B2ST,B3ST,C1ST,C2ST]

AE = []
AE_l = []
AE_names = []
AE_namesd = []
for iw,w in enumerate(analyticalValues):
    AE_names.append(str(w.__name__)[:2])
    for j in range (0,k):
        AE.append(w(j))
        AE_namesd.append('AE' + str(w.__name__) + str(j))
AE_df = pd.DataFrame([AE[k*iw:k*(iw+1)] for iw in range(len(analyticalValues))],AE_names)
AE_dic = create_dict(AE_namesd, AE)

### It is then time to define the sample and scrambled matrices

In [ ]:
p = 13
run = 50

n = [3,4,6]

df = pd.DataFrame(sobol_seq.i4_sobol_generate(6*k, run*2**p))

### And to assess the value of the MAE for the test functions

In [ ]:
for in1,n1 in enumerate(n):
    run_samples = []

    MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
    for r in range (run):
        run_samples.append(df.iloc[int(r*(len(df)/(run*2**(in1+2)))):int((r+1)*(len(df)/(run*2**(in1+2))))].reset_index(drop=True))

        sample_Matrices = [run_samples[-1].iloc[:,m*k:(m+1)*k].T.reset_index(drop=True).T for m in range(n1)]

        mixed_Matrices = []
        elementary_effects_list =[]
        f_elementary = []
        for im,m in enumerate(range(n1)):
            elementary_effects_dic = {j:[] for j in range(k)}
            f_elementary.append({f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions})
            for j in range(k):
                elementary_effects = [sample_Matrices[m]]
                for q in range(n1-1):
                    mixed_Matrices.append(sample_Matrices[m].copy())
                    mixed_Matrices[-1][j]=sample_Matrices[np.roll(np.arange(n1),(n1-1)*m)[q+1]][j]
                    
                    elementary_effects.append(mixed_Matrices[-1])
                elementary_effects_dic[j]=elementary_effects
                
            elementary_effects_list.append(elementary_effects_dic)
            
        for f in functions:
            for ie,e in enumerate(elementary_effects_list):
                for j in range(k):
                    el = []
                    for q in range(len(e[j])):
                        for qi in range(q+1,len(e[j])):
                            el.append(0.5*(f(e[j][q])-f(e[j][qi]))**2)
                    f_elementary[ie][f.__name__][j] = pd.concat(el)
                C_T = pd.concat([fe[f.__name__] for fe in f_elementary]).sort_index().expanding(1).mean()
            Var = f(pd.concat(sample_Matrices).sort_index()).expanding(1).var(ddof=0).expanding(1).mean()
            T = (C_T.iloc[::int(n1*n1*(n1-1)/2)].T/Var.iloc[::n1]).T
            AE_r = np.abs(T - AE_df.loc[f.__name__])
            MAE_r = AE_r.mean(axis=1)
            MAE_r.index=(MAE_r.index+1)*len(sample_Matrices)*(1+k*(len(sample_Matrices)-1))
            MAE_dic[f.__name__][r]=MAE_r
    writer = pd.ExcelWriter(str(n1)+'Multiple_matrices.xlsx', engine='xlsxwriter')

    for mk in MAE_dic.keys():
        MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)
    
    writer.save()

In [ ]:
C_T